In [1]:
import pandas as pd
import os
import re

In [7]:
# CALCULATE BUDGETS FOR EACH AREA
# link to source: https://app.clarity.so/rawdao/work/6

init_budget = {}

init_budget['governance'] = 1.5
init_budget['finance'] = 1.5
init_budget['curation'] = 2.5
init_budget['internal relations'] = 1
init_budget['marketing & pr'] = 1

perf_tranche = {}

perf_tranche['governance'] =  0
perf_tranche['finance'] = 0
perf_tranche['curation'] = 0
perf_tranche['internal relations'] = 0
perf_tranche['marketing & pr'] = 0

perf_score = {}

perf_score['governance'] =  0
perf_score['finance'] = 0
perf_score['curation'] = 0
perf_score['internal relations'] = 0
perf_score['marketing & pr'] = 0


final_budget = {}
sum = 0

for area in init_budget:
  final_budget[area] = init_budget[area]*(1-perf_tranche[area]) + init_budget[area]*(perf_tranche[area]*perf_score[area])
  sum = sum + final_budget[area]

print(final_budget)
print(sum)

{'governance': 1.5, 'finance': 1.5, 'curation': 2.5, 'internal relations': 1, 'marketing & pr': 1}
7.5


In [24]:
# READ AND CLEAN COORDINAPE DATA
dfs = {}
voting_members = {}
total_give = {}

for file in os.listdir('./coordinape_data'):
  try:
    # read data from coordinape files  
    key = re.search('DAO-(.+?)-', file).group(1).lower()
    # print(key)
    dfs[key] = pd.read_csv('./coordinape_data/'+ file)
    
    # calculate voting stats for each area
    voting_members[key] = int(dfs[key]['sent'].astype(bool).sum(axis=0))
    total_give[key] = int(dfs[key]['received'].sum())

    # calculate compensation per area
    dfs[key] = dfs[key].sort_values(by=['received'], ascending = False) # sort
    dfs[key] = dfs[key][['name','address','received']] # select relevant cols
    dfs[key]['percent_comp'] = dfs[key]['received']/total_give[key] # calculate percentage comp.
    dfs[key]['eth_compensation'] = dfs[key]['percent_comp'] * final_budget[key] # calculate actual ETH comp.
    dfs[key] = dfs[key][dfs[key]['eth_compensation'] > 0] # filter people with 0 compensation
    dfs[key]['area'] = key
    dfs[key]['total_budget_of_area'] = final_budget[key]
  except NameError:
    print(file + ' -> file skipped')
    print(key)
    print(NameError)

# prepare area breakdown for export
df_export = pd.concat(dfs)

# merge everything and calculate compensation per member
comp_df = pd.concat(dfs)
comp_df = comp_df[['name','address','eth_compensation']]
comp_df = comp_df.groupby(by=['name','address']).sum()
comp_df = comp_df.sort_values(by=['eth_compensation'], ascending = False)


print(voting_members)
print(total_give)

comp_df

{'curation': 18, 'finance': 15, 'governance': 15, 'internal relations': 13, 'marketing & pr': 21}
{'curation': 1676, 'finance': 1328, 'governance': 1256, 'internal relations': 1152, 'marketing & pr': 1906}


,,eth_compensation
name,address,
lucaspon,0x586bc43937c2ec42348cc83acf44ced42fe3d5f7,1.295928
Lapsus#4632,0x96e1f3e799133567258871cebff1760e4b4a655f,0.902047
hddamico#7928,0x5c2a00d4051455f05829d818dac18e438124e79c,0.591126
claudia#0273,0x71b49fe06436d997b14228c96ac9bf4439147259,0.478836
alexcartagenamex#4215,0x7bc7481751d7fcb76d08f596b03ba1e42378ea27,0.417482
shanelavalette#2830,0x4ad06a01c14cb8cf20e49fc53c647b882c1628e9,0.316618
glory#5406,0x14df40586e0f2ca5ca222db0dac8df2f4c5b1aab,0.310769
chadwicktyler#3331,0x3d66c6ba3ce3fdc988654d72ed83a478851fde49,0.287992
mmiliszkiewicz#5782,0x1255b354efdf56e62fc70b0f0b155d026b324626,0.252136


In [17]:
# EXPORT RESULTS TO EXCEL
from pathlib import Path
Path("./outputs").mkdir(parents=True, exist_ok=True)

df_export.to_excel("./outputs/compensation_per_area.xlsx")
comp_df.to_excel("./outputs/compensation_by_member.xlsx")